# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

Automated facial expression recognition provides an objective assessment of emotions. Human based assessment of emotions has many limitations and biases and automated facial expression technology has been found to deliver a better level of insight into behavior patterns. Emotion detection from facial expressions using AI is useful in automatically measuring consumers’ engagement with their content and brands, audience engagement for advertisements, customer satisfaction in the retail sector, psychological analyses, law enforcement etc.

In [1]:
#@title Explanation Video
from IPython.display import HTML

HTML("""<video width="500" height="300" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Hackathon3b_expression_recognition.mp4" type="video/mp4">
</video>
""")

**Objectives:** 

**Stage 4 (15 Marks):** Train a CNN Model and perform Expression Recognition in the EFR Mobile App.

**Stage 5 (5 Marks):** Test for Anti-Face Spoofing on the EFR Mobile App.

##**Stage 4 (15 Marks)**

**(i) Train a CNN Model for Expression Recognition on given Expression data  
(ii) Deploy the Model and Perform Expression Recognition on Team Data through the EFR Mobile App**


---


* Define and train a CNN for expression recognition for the data under folder "Expression_data" which segregated on expression basis.
* Collect your team data using EFR application and test your model on the same and optimize the CNN architecture for predicting the respective labels of the images.
* Save and Download the trained expression model and upload them in the ftp server (refer to [Filezilla Installation and Configuration document](https://drive.google.com/file/d/1hnMXcwpCwAz94ljAhtsJdQSfe-JaOD5K/view?usp=sharing)).

* Update the **“exp_recognition.py”** file in the server. Open the files in the terminal (Command prompt) and provide the code for predicting the expression on the face (Note: To define the architecture of your trained model, you'll need to define it in the file **"exp_recognition_model.py"**). 

* Test your model on the mobile app for Expression Recognition and Sequence Expression. Your team can also see your results in your terminal.


* Grading Scheme:
> * Expression Recognition (12M): If the functionality is returning expression class correctly for the face using the mobile app’s “Expression Recognition” functionality
> * Sequence Expression (3M): Get three consecutive correct Expressions using the mobile app’s “Sequence Expressions” functionality

**Download the dataset**

In [1]:
#@title Run this cell to download the dataset

from IPython import get_ipython
ipython = get_ipython()
  
notebook="M3_Hackathon" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Expression_data.zip")
    
    ipython.magic("sx unzip Expression_data.zip")
    
    ipython.magic("sx pip install torch==1.0.1 -f https://download.pytorch.org/whl/cu100/stable")
    ipython.magic("sx pip install torchvision==0.2.1")
    ipython.magic("sx pip install opencv-python")
    print ("Setup completed successfully")
    return
setup()

Setup completed successfully


**Dataset attributes:**

During the setup you have downloaded the Expression data:

* **Expression_data**: In this folder, the images are segregrated in terms of Expression
> * Expressions available: ANGER, DISGUST, FEAR, HAPPINESS, NEUTRAL, SADNESS, SURPRISE
> * Each class is organised as one folder
> * There are ~18000 total images in the training data and ~4500 total images in the testing data

In [2]:
%ls

Expression_data/  Expression_data.zip  __MACOSX/  sample_data/


**Imports: All the imports are defined here**

We are installing the following specific package versions -> torch 1.0.1, torchvision 0.2.1 and PIL 5.3.0 to maintain compatibility with the server 

* Firstly uninstall and downgrade the current PIL version. In the next cell, you will see a button "Restart Runtime" button appear below. 
* Click on it and select 'Yes' to restart runtime and reset the PIL package. 
* **DO NOT** go to the notebook's **RUNTIME  -> RESTART RUNTIME**. This will restart all packages and you will need to repeat all the steps from beginning.


* Simply continue with the next code cell

PIL (Pillow) is the Python Image Library. Used to cut and resize images, or do simple manipulation.


In [3]:
!pip uninstall -y Pillow

Uninstalling Pillow-7.0.0:
  Successfully uninstalled Pillow-7.0.0


In [1]:
# IGNORE ERROR. Click on Restart Runtime button and slect 'Yes' if prompts. Then proceed with the next code cell.
!pip install Pillow==5.3.0

In [2]:
# When you run this, it should give you pil version = 5.3.0
import PIL
print(PIL.__version__)

5.3.0


In [3]:
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os
import warnings
from time import sleep
import sys
warnings.filterwarnings('ignore')

For the following step, to obtain hints on building a CNN model for face expression, you may refer to this [article](https://drive.google.com/open?id=1P2rpaWW3tOtGGnw4dvtdZ4hjoc8iDNst)

**Define and train a CNN model for expression recognition**

In [4]:
transform_train = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomRotation(20),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485], std=[0.229])])

transform_validate = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485], std=[0.229])])

In [16]:
trainset = dset.ImageFolder(root='./Expression_data/Facial_expression_train/', transform=transform_train)
testset = dset.ImageFolder(root='./Expression_data/Facial_expression_test/', transform=transform_validate)

In [25]:
trainloader = DataLoader(trainset,batch_size=16,shuffle=True)
testloader = DataLoader(testset,batch_size=16,shuffle=False)

In [26]:
import torch
import torch.nn as nn
import torchvision.models as models


In [39]:
# YOUR CODE HERE to define and train CNN model for Expression_Data.

model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = True

model.fc = nn.Sequential(nn.Linear(2048, 256), 
                        nn.ReLU(), 
                        nn.Linear(256, 100), 
                        nn.ReLU(),
                        nn.Linear(100, 7))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


In [40]:
from torchsummary import summary

In [41]:
print(summary(model.cuda(),input_size=(3,224,224)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [42]:
#training loop
epochs = 5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Training on Device: {device}")
for epoch in range(epochs):
    running_loss = []
    
    model.train()
    model.to(device)
    n=0
    for idx,(images,labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        n+=1
        optimizer.zero_grad()
        #print(item)
        output = model(images)
        loss = criterion(output, labels)
        
        loss.backward()
        if idx%16 == 0: 
          print(loss.item(), epoch) 
        running_loss.append(loss.item())
        
        optimizer.step()
        
        #validation loop
    with torch.no_grad():
        model.eval()
        total = 0
        correct = 0
        for images,labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            _, pred = torch.max(output.data, 1)
            
            total+= labels.size(0)
            correct += (pred == labels).sum().item()
            
        print(f"Epoch: {epoch}")
        print(f"Training Loss: {sum(running_loss)/len(running_loss)}")
        print(f"Validation Accuracy: {float(correct)/total}")

Training on Device: cuda
1.9403088092803955 0
1.8676435947418213 0
1.792262315750122 0
1.7722524404525757 0
1.7476600408554077 0
1.496900200843811 0
1.8165913820266724 0
1.6312487125396729 0
1.6125746965408325 0
1.6983646154403687 0
1.9816495180130005 0
1.554158091545105 0
1.8170920610427856 0
1.8561561107635498 0
1.8653318881988525 0
1.4786951541900635 0
2.0035245418548584 0
1.6071925163269043 0
1.7542295455932617 0
1.4954873323440552 0
1.8878943920135498 0
1.6491085290908813 0
1.6459287405014038 0
1.7156982421875 0
1.598759412765503 0
1.648256540298462 0
2.0250775814056396 0
1.77474045753479 0
1.6115318536758423 0
1.4083822965621948 0
1.6528981924057007 0
1.5956058502197266 0
1.5850547552108765 0
1.5772109031677246 0
1.9190353155136108 0
1.7772668600082397 0
1.519682765007019 0
1.710207462310791 0
1.8436899185180664 0
1.9460381269454956 0
1.6476550102233887 0
1.9086713790893555 0
1.7630562782287598 0
1.847900152206421 0
1.6373794078826904 0
1.873079776763916 0
1.7054660320281982 0
1.

**Test your model and optimize CNN architecture for predicting the labels correctly**

In [ ]:
# YOUR CODE HERE for test evaluation

**Team Data Collection (activate the server first)** 

  - (This can be done on the day of the Hackathon once the login username and password are given)

Activate the Server Access
* Open the terminal (Command Prompt)
* Login to SSH by typing **ssh (username)@aiml-sandbox1.talentsprint.com**. Give the login username which is given to you. 

Eg: `ssh b15h3gxx@aiml-sandbox1.talentsprint.com`

  (If it is your first time connecting to the server from this computer, accept the connection by typing "yes".)
* After logging into SSH, please activate your virtual environment using the
command **source venv/bin/activate** and then press enter
* You can start the server by giving the command **sh runserver.sh** and then press enter.
* In order to collect team data in mobile app, ensure the server is active


**Collect your team data using the EFR Mobile App and fine-tune the CNN for expression data on your team**

Team Data Collection

* Follow the "Mobile_APP_Documentation" to collect the Expression photos of your team. These will be stored in the server to which login is provided to you.

[Mobile_APP_Documentation](https://drive.google.com/file/d/1tpr8_U0Ll_TexN4s-0pmPPg23J7Usok2/view?usp=sharing)


**Download your team expression data from the EFR app into your colab notebook using the links provided below**

NOTE: Replace the string "username" with your login username (such as b15h3gxx) in the below cell for expression images. 

This data will be useful for testing the above trained cnn networks.

In [43]:
!wget -nH --recursive --no-parent --reject 'index.*' https://aiml-sandbox.talentsprint.com/expression_detection/b15h3g12/captured_images_with_Expression/ --cut-dirs=3  -P ./captured_images_with_Expression

--2021-02-20 09:42:00--  https://aiml-sandbox.talentsprint.com/expression_detection/b15h3g12/captured_images_with_Expression/
Resolving aiml-sandbox.talentsprint.com (aiml-sandbox.talentsprint.com)... 139.162.203.12
Connecting to aiml-sandbox.talentsprint.com (aiml-sandbox.talentsprint.com)|139.162.203.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./captured_images_with_Expression/index.html.tmp’

index.html.tmp          [ <=>                ]   1.03K  --.-KB/s    in 0s      

2021-02-20 09:42:01 (117 MB/s) - ‘./captured_images_with_Expression/index.html.tmp’ saved [1054]

Loading robots.txt; please ignore errors.
--2021-02-20 09:42:01--  https://aiml-sandbox.talentsprint.com/robots.txt
Reusing existing connection to aiml-sandbox.talentsprint.com:443.
HTTP request sent, awaiting response... 404 Not Found
2021-02-20 09:42:01 ERROR 404: Not Found.

Removing ./captured_images_with_Expression/index.html.tmp since it should 

In [ ]:
%ls

In [ ]:
# YOUR CODE HERE for loading the team expression data. Note: Use the same transform which used for Expression_Data.
# YOU CODE HERE for Dataloader
finalClassifierDset = dset.ImageFolder(root='./captured_images_with_Expression',
                                       transform = transforms.Compose([transforms.Grayscale(num_output_channels = 1), transforms.Resize((100,100)), transforms.ToTensor()]))
representation_dataloader = DataLoader(finalClassifierDset, shuffle=False, num_workers=8, batch_size=100)

In [ ]:
# YOUR CODE HERE for getting the CNN representation of your team data with expression. Optimize the CNN model for predicting the labels of expressions correctly


**Save your trained model**

* Save the state dictionary of the classifier (use pytorch only), It will be useful in
integrating model to the mobile app

 [Hint](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

In [44]:
### YOUR CODE HERE for saving the CNN model
state = {'net_dict': model.state_dict()}
torch.save(state, './resnet50_model.t7')

**Download your trained model**
* Given the path of model file the following code downloads it through the browser

In [45]:
from google.colab import files
files.download('./resnet50_model.t7')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##**Stage 5 (Anti Face Spoofing): (5 marks)**


---



The objective of anti face spoofing is to be able to unlock (say) a screen not just by your image
(which can be easily be spoofed with a photograph of yours) but by a switch in the expression
demanded by the Mobile App (which is much less probable to mimic)
* **Grading scheme**:
> * **Anti Face Spoofing**: (5M Only if both the cases mentioned below are achieved)
>>* **Unlock**: Correct face + Correct Demanded Expression
>>* **Stay Locked**: Correct face + Incorrect Demanded Expression (as you might imagine there are multiple other such possibilities, which you are free to explore)

In [ ]:
# Test in your mobile app and see if it gets unlock.